## Advanced queries

In the previous section we saw only the basics of queries. This time we'll be going through how to create queries for lists, nested documents and the special case of queries by coordinates.

In this section we'll be using a sample of ALeRCE-like objects that should be loaded into the database if the instructions in `README.md` have been followed.

**Note:** The objects in the actual ALeRCE database do not exactly follow this structure. We're using only a simplified version here to show these concepts. For an example document in this collection:

In [ ]:
from pprint import pprint
from pymongo import MongoClient

client = MongoClient(host='localhost', port=27017, username='mongo', password='mongo')

objects = client.alerce.objects  # This is the collection we'll be using

pprint(objects.find_one())

### Array and nested document queries

As you can see, the fields `oid` and `tid` are lists of strings (called arrays in MongoDB), while `loc` is a nested document with a list within. Also, `probabilities` is an array of nested documents. 

In the previous section we saw how to search by fields that are numeric or strings. Most operators are similar, but there's a specific syntaxis (and a few special operators) for querying over elements of arrays and nested documents.

**Note:** In these examples we'll use `find_one`, but everything seen here also works for `find`.

To begin simply, it is always possible to search by exact match of an array:

In [ ]:
objects.find_one({'oid': ['ZTF17aaaacji']})  # Note the []

The above query only matches if `oid` is exactly the same list provided, including the number of elements and order. Even if another document were to contain the string `ZTF17aaaacji` in `oid`, but with more than one element, it would not match.

For a query that checks if an element is in the array, independently of its position and the number of elements in the array, the following can be used: 

In [ ]:
objects.find_one({'oid': 'ZTF17aaaacji'})  # This time there is no []

The above is an equivalent but more compact form of using the operator `$in` with a single element:

In [ ]:
objects.find_one({'oid': {'$in': ['ZTF17aaaacji']}})  # The value for $in can have more than one element

It is also possible to query by the location on the array. It is possible to apply filters to a given element using the name of the array field with dot notation for the index of the desired element in the array (*MongoDB uses zero based index*). For instance, the query below only applies to the first element of the `oid` array, even if other object has a second element with that value, it will not match our query:

In [ ]:
objects.find_one({'oid.0': {'$regex': '^ZTF17a'}})  # Starts with ZTF17a

To query over nested fields, we also use the dot notation, but this time using the name of the nested field:

In [ ]:
objects.find_one({'loc.type': 'Point'})

In the case of an array inside a nested document, everything we saw above works as normal. For instance, to access an element by index, the dot notation can be continued:

In [ ]:
objects.find_one({'loc.coordinates.1': {'$gt': 52}})  # Second element

Finally, something similar applies to a list of nested documents:

In [ ]:
objects.find_one({'probabilities.0.probability': {'$gt': 0.5}})  # Probability of first element above 0.5

**Note:** In any query over arrays, nested documents or a combination thereof, the full array/nested document will be returned, not just the matching elements.

However, there is an additional way to search over a nested field as an independent list. For instance to check that at least one probability (in any position) is above 0.8:

In [ ]:
objects.find_one({'probabilities.probability': {'$gt': 0.8}})  # Note the lack of the dot notation for index

**Note:** Remember that in MongoDB the documents do not necessarily have to have the same fields. This also applies to nested documents. When using something like the above, it will not check over nested documents that lack the `probability` field.

There are a few operators that work only with arrays. We've seen `$in` already, but we'll see a few others now.

To begin with, there is the `$all` operator. This works in a similar manner to `$in`, but instead of just requiring that at least one element matches, it requires that all listed elements are present in the array (independently on the order or presence of additional elements):

In [ ]:
objects.find_one({'loc.coordinates': {'$all': [52.2932219572289, -112.883127892771]}})

For arrays of nested documents, it is important to note that trying to match over each element might result in unintuitive matches. In the following, one might expect to receive objects where the class `CV,Nova` has a probability above 0.8. However, this is not the case:

In [ ]:
objects.find_one({'probabilities.probability': {'$gt': 0.8}, 'probabilities.class_name': 'CV,Nova'})

What happened? The restrictions of the query each apply independently. It matches any objectes where there is at least an element of `probability` over 0.8, and at least one element of `class_name` with value `CV,Nova`. The two of them have to match, but there is no requirement for the same element to match all of them.

To ensure all conditions are matched by at least one element simultaneously, we have to use the `$elemMatch` operator:

In [ ]:
# Note that we query over the full array and inside $elemMatch we access subfields directly, without dot notation
objects.find_one({'probabilities': {'$elemMatch': {'probability': {'$gt': 0.8}, 'class_name': 'CV,Nova'}}})

### Expression queries

Another types of queries allows for the use of what are called expressions. These are typically some sort of operation over the fields, rather than a direct search over the values. For instance, to retrieve objects with difference between last and first detection dates of less than a thousand days, we would use: 

In [ ]:
objects.find_one(
    {
        '$expr': {
            '$lte': [{'$subtract': ['$lastmjd', '$firstmjd']}, 1000]
        }
    }
)

The `$expr` operator can use withing the standard query operators (`$lte` in this case), but now with a value that is a list with the field in the first position, which can now be an expression (in this case `$subtract`) and the second value given for the comparison. Note that the fields used within the expression *must* be preceeded by `$`. A list of all expression operators can be found [here](https://www.mongodb.com/docs/v6.0/meta/aggregation-quick-reference/#operator-expressions).

Expressions are an important concept, separated from operators. Some operators can receive expressions and some expressions can have the same name as operators, but with different usage. *Always check the documentation.*

### Geospatial queries

MongoDB has implemented geospatial queries. Originally intended to be used for terrestrial coordinates, they can be used as well for celestial coordinates (although keeping in mind some quirks and exceptions). While multiple types of searches are possible depending on the geometries defined, we will focus only on cone-searches, which are the more relevent for usage within ALeRCE. For other types of geospatial queries, see [here](https://www.mongodb.com/docs/manual/reference/operator/query-geospatial/).

The field `loc` has the form we've seen before so that it can be used for geospatial queries. The first value of `coordinates` corresponds to the right ascension minus 180, while the declination remains the same. This is because a document with `type` and `coordinates` is defined by [GeoJSON](https://www.mongodb.com/docs/manual/reference/geojson/) and normally used for latitude/longitude coordinates (thus why we need the "longitude" to go from -180 to 180).

Now, to search for elements within a circle over the sphere we use the operator `$geoWithin`. Inside the operator there are multiple options that can be used, but in the case of the circle we use `$centerSphere`, which has as value an array. The first element is an array of coordinates (longitude and latitude, or right ascension minus 180 and declination in our case, in degrees) and the second element is the radius (in radians):

In [ ]:
objects.find_one(
    {
        'loc': {
            '$geoWithin': {
                '$centerSphere': [[67 - 180, 52], 3.14 / 180]  # Within one degree (approx)
            }
        }
    }
)

It is also possible to use other geometries, for instance a `$box`. This uses an array with two arrays, representing oposite corners of the box:

In [ ]:
objects.find_one(
    {
        'loc': {
            '$geoWithin': {
                '$box': [[67 - 180, 52], [68 - 180, 53]]
            }
        }
    }
)

The operator `$geoWithin` will return the objects within the given geometry with no particular order, i.e., the first result might not be the one nearest to the center.

## Indexing

Indexing is used to make searches more efficient. When there are no indices, any search goes through each full document in order to find the specified results. If a search uses indexed fields, it can perform more efficiently by going only through the indexed field(s), and then retrieving only the matching documents without having to go through the full collection. However, not everything needs to be indexed, since the storage usage can grow substantially when too many indexes are created. It is generally recommended to limit the number of indices to the most commonly used fields.

For the examples in this tutorial, neither the gain in search speed nor the extra storage required will be noticeable, but for large collections this has to be taken into account.

Every collection in MongoDB comes indexed by `_id` by default, but new indices can be created at any time and there are different kinds of indices.

**Note:** If your user doesn't have write permission, you won't be able to create your own indices, but the gains when doing queries over existing indices will still be noticeable.

### Creating basic indices

To create a simple index over a field, the structure is similar to what we saw for the `sort` option in the `find` method. It should be a list of pairs with a direction given by `1` or `-1` (ascending or descending, respectively).

In [ ]:
from pymongo import IndexModel

objects.create_index([('firstmjd', 1)])

The return value correspond to the name given to the index in the collection.

In [ ]:
objects.index_information()

The indices here are given by index name and have information about the field, order and type of index (the `v` field refers to the version of indexing).

When performing queries, it is possible to check if indices are being used. For this, there is the `explain` method for `Cursor`, which returns a dictionary with information regarding the query:

In [ ]:
info = objects.find({'firstmjd': {'$gte': 58400}}).explain()
exec_stats = info['executionStats']

print(f'Docs returned: {exec_stats["nReturned"]}')
print(f'Keys checked : {exec_stats["totalKeysExamined"]}')
print(f'Docs checked : {exec_stats["totalDocsExamined"]}')
print(f'Time (ms)    : {exec_stats["executionTimeMillis"]}')

There is a lot more infomation from `explain`, but we'll center on the relevant ones for this section (all within `executionStats`):

* `nReturned`: Number of documents returned
* `totalKeysExamined`: Number of indexed fields checked during the query
* `totalDocsExamined`: Number of documents checked during the query
* `executionTimeMillis`: Execution time in milliseconds. This is always an integer and is rounded

For comparison, let's do a query over a non-indexed field:

In [ ]:
info = objects.find({'lastmjd': {'$gte': 59550}}).explain()
exec_stats = info['executionStats']

print(f'Docs returned: {exec_stats["nReturned"]}')
print(f'Keys checked : {exec_stats["totalKeysExamined"]}')
print(f'Docs checked : {exec_stats["totalDocsExamined"]}')
print(f'Time (ms)    : {exec_stats["executionTimeMillis"]}')

Even though it actually returns less documents, the query checks all documents in the collection, without checking any key (as there is no indexing over `lastmjd`). In this case the execution time is still below a millisecond, but for large collections, such full scans can make a big difference.

When using indices, the number of keys checked are usually in the order of the returned documents, although some variations depending on the query and ordering can alter that. Normally as well, it only checks the documents that it needs to return, making it most of the time the most efficient option. The only exception is for queries that return most of the collection, as then it will scan the indices and then the documents themselves, but the losses here are generally minor compared with gains over more restricted queries.

**`mongosh`:** It is also possible to use `explain` in the Shell:

```bash
   db.objects.find({lastmjd: {$gte: 59550}}).explain()
```

One can also drop an index based on the index name:

In [ ]:
objects.drop_index('firstmjd_1')
objects.index_information()

At creation, an index can also be given a custom name, instead of relying on MongoDB for the name:

In [ ]:
objects.create_index([('firstmjd', 1)], name='first_detection_date')

In the case of the field `probabilities`, we have the case of a an array with nested documents. If we wanted to create an index for a field inside it, we can use dot notation:

In [ ]:
objects.create_index([('probabilities.probability', -1)], name='probs')

As for other common options, it is possible to demand that the elements of the index are unique:

In [ ]:
objects.create_index([('oid', 1)], unique=True)

Or creating a partial index (only indexes documents that fullfill a given condition):

In [ ]:
objects.create_index([('lastmjd', 1)], partialFilterExpression={'ndet': {'$gt': 100}})

The above creates an index over `lastmjd`, but only for documents with `ndet` greater than 100.

**`mongosh`:** As we've seen for sorting, the main difference here is that index pairs are passed as dictionaries (an the options are also passed differently):

```bash
   db.objects.createIndex({lastmjd: 1}, {partialFilterExpression: {ndet: {$gt: 100}}})
```

For now, we'll just remove all indices (note that this will never remove the index over `_id`):

In [ ]:
objects.drop_indexes()

### Geospatial indexing

Some fields can have a special indexing, such as the coordinates over a sphere. These use the special ordering `2dsphere` and must have a form compatible with GeoJSON:

In [ ]:
objects.create_index([('loc', '2dsphere')])

If such an index is used, it is possible to do a cone-search where the results are ordered from nearest to furthest from the center (unlike `$geoWithin`). In this case we must use the operator `$nearSphere` with `$maxDistance`:

In [ ]:
objects.find_one(
    {
        'loc': {
            '$nearSphere': [67 - 180, 52],  # Center of circle
            '$maxDistance': 3.14 / 180.  # Within one degree (approx)
        }
    }
)

Optionally, it is also possible to provide a `$minDistance` to exclude results too close to the center:

In [ ]:
objects.find_one(
    {
        'loc': {
            '$nearSphere': [67 - 180, 52],  # Center of circle
            '$minDistance': 0.001,  # Min distance in radians
            '$maxDistance': 3.14 / 180.  # Within one degree (approx)
        }
    }
)

**Note:** There is also another usage of `$nearSphere` that involves giving the distances in meters. This is intended for use with geographical coordinates, as it takes into account the deviations of Earth's shape from a perfect sphere. We won't go through this usage as it is outside the scope, but more details can be found [here](https://www.mongodb.com/docs/v6.0/reference/operator/query/nearSphere/#mongodb-query-op.-nearSphere). In short, if the value of `$nearSphere` is a `geometry` rather than a list with coordinates, the minimum and maximum distances are assumed to be in meters. Be mindful of not confusing both usages.

**Note:** The use of sorting when querying using `$nearSphere` will override the order in which the objects are returned. In most cases, if a sort is to be performed, it is preferable to use the operator `$geoWithin`.

### Compund indices

It is also possible to create indices over multiple fields at a time, with them being sorted by in order (latter indices fixing clashes over the first). This can be very useful depending on the type of search. For instance, we'll create an index over the classifier name and version, so that all versions are in order, but first sorted by classifier (for each document):

In [ ]:
objects.create_index([('probabilities.classifier_name', 1), ('probabilities.classifier_version', 1)])  # The list now has a second element (the secondary index)

Again we'll clean up all the indices:

In [ ]:
objects.drop_indexes()

## Projections (customizing output documents)

The `find` and `find_one` methods have some additional functionality that we'll discuss now. Besides the dictionary with the query filters, a second argument can be passed with another dictionary for "projection". These projections allow for different manipulations of the output documents, *without modifying them on the database*. These can range from selection which fields are to be returned, all the way to creating new fields based on existing data.

For instance, if there is only one field of interest in the output:

In [ ]:
docs = objects.find({'ndet': {'$gte': 100}}, {'firstmjd': True, 'lastmjd': True})

for doc in docs:
    print(doc)

**`mongosh`:** It is also possible to use projections in an equivalent manner:

```bash
   db.objects.find({ndet: {$gte: 100}}, {firstmjd: true, lastmjd: true})
```

**Note:** Indices are only used for the query itself. Projections do not benefit from indexing.

Here we've selected only objects with more than 100 detections, but are only interested in the first and last MJD. As you can see, even if not explicitly selected, the `_id` field will be carried by default. This behaviour can be changed:

In [ ]:
docs = objects.find({'ndet': {'$gte': 100}}, {'firstmjd': True, 'lastmjd': True, '_id': False})

for doc in docs:
    print(doc)

If at least one of the projected fields is explicitly selected, all the other will be implicitly removed. The oposite is also true:

In [ ]:
docs = objects.find({'ndet': {'$gte': 100}}, {'probabilities': False, '_id': False})

for doc in docs:
    print(doc)

The above includes every field, except for `probabilities` and `_id`. Note that it is not possible to mix inclusion and exclusion of fields, except for the case of `_id`:

In [ ]:
# Will fail due to mixing inclusion and exclusion
docs = objects.find({'ndet': {'$gte': 100}}, {'probabilities': False, 'firstmjd': True})
# However, it will only fail at this stage
for doc in docs:
    print(doc)

It is possible also to change the names of fields using the new name as key and the old name as value (with `$` before the name, given that this is technically an expression):

In [ ]:
docs = objects.find({'ndet': {'$gte': 100}}, {'detections': '$ndet', 'firstmjd': True})

# Renamed ndet to detections
for doc in docs:
    print(doc)

It is also possible to project embedded documents using also expressions:

In [ ]:
docs = objects.find({'ndet': {'$gte': 100}}, {'probability': '$probabilities.probability', '_id': False})

# Renamed ndet to detections
for doc in docs:
    print(doc)

Expressions can also be more generally used in projections. Essentially, we can create custom fields:

In [ ]:
# Field deltamjd is going to be subtration of two original fields
docs = objects.find(
    {}, 
    {'deltamjd': {'$subtract': ['$lastmjd', '$firstmjd']}, '_id': False}
)

for doc in docs:
    print(doc)

The projections can also limit the number of elements returned from an array. Using the dot notation with `$`, we'll only keep the first element that matches the original (other matches will be ignored):

In [ ]:
docs = objects.find({'probabilities.ranking': 1}, {'probabilities.$': True, '_id': False})

for doc in docs:
    print(doc)

The `$` operator seen above will only return the first element of the array that matches the query,  even if more than one element does. For this reason it requires for the array to actually be used within the query.

For more control over the returned element, there is also the `$elemMatch` projection operator:

In [ ]:
docs = objects.find(
    {
        'ndet': {'$gte': 100}
    }, 
    {
        '_id': False,
        'probabilities': {
            '$elemMatch': {  # The value for $elemMatch has the form of a query, over the fields inside the elements
                'classifier_name': 'stamp_classifier',
                'ranking': 1
            }
        },
    })

for doc in docs:
    print(doc)

**Note:** `$elemMatch` for projection and queries function differently, they are actually considered different operators and their behaviour is related to their context. For queries it makes sure that at least one element matches all criteria. In the case of the projection, it will only return the *first* element that the criteria (or an empty list if none matches), any other element matching the criteria will be silently dropped.

In order to retrieve all matching elements, not just the first, it is better to use `$filter`, but note that it has a different usage:

In [ ]:
docs = objects.find(
    {
        'ndet': {'$gte': 100}
    }, 
    {
        '_id': False,
        'probs': {  # This is the name of the output array (can be anything)
            '$filter': {
                'input': '$probabilities',  # This is the name of the input array
                'cond': {  # The condition that needs to match
                    '$and': [
                        {'$eq': ['$$this.ranking', 1]},
                        {'$eq': ['$$this.classifier_name', 'stamp_classifier']}
                    ]
                }
            }
        },
    })

for doc in docs:
    print(doc)

**Note:** The operator `$filter` is only available for MongoDB version 3.2 or above.

The `$filter` operator has the following fields: 

* `input` must be an expression that returns an array, thus why we use the `$` at the beginning of the field name (could also be something more complex).
* `cond` must also be an expression.

The name `$$this` refers to elements of the array defined in `input`. It is possible to change the name from `this` to something else using the (optional) field `as`:

In [ ]:
docs = objects.find(
    {
        'ndet': {'$gte': 100}
    }, 
    {
        '_id': False,
        'probs': {
            '$filter': {
                'input': '$probabilities',
                'as': 'element',  # New name for items
                'cond': {
                    '$and': [
                        {'$eq': ['$$element.ranking', 1]},  # now using 'element' instead of 'this'
                        {'$eq': ['$$element.classifier_name', 'stamp_classifier']}
                    ]
                }
            }
        },
    })

for doc in docs:
    print(doc)

## Summary

New concepts seen here:

* Indices allow for better performance over queries that involve indexed fields, preventing a full collection scan
* Projections allow us to limit or rename the number of fields returned
* Expressions allow for more complex queries, not just direct comparissons with the values, but also over operations involving one or more fields
  
Things to keep in mind:

* Some operators might ignore certain types of indices. If you see a loss of performance, check the `explain` method and documentation to make sure things are being used as expected
* Queries over arrays are sometimes unintuitive. To ensure that the query matches for a single element, use `$elemMatch`. Otherwise the match might be over different elements
* Geospatial indices and searches works somwhat differently than others and the latter cannot be done without the existence of the former